This notebook works through a set of tests to calculate the weights daymet contributing cells to each hru in GFv11

In [1]:
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

import geopandas as gpd
import pandas as pd
import glob
import zipfile
import rasterio
import os
import xarray as xr
import json

Populating the interactive namespace from numpy and matplotlib


In [2]:
print(os.getcwd())
from pathlib import Path
folder = Path(r'../Data') # assumes working directory is onhm-fetcher-parser
print(folder)
shapefiles = folder.glob("*_0[1].shp")
#shapefiles = folder.glob("*.shp")
gdf = pd.concat([
    gpd.read_file(shp)
    for shp in shapefiles
]).pipe(gpd.GeoDataFrame)
gdf.reset_index(drop=True, inplace=True)
# gdf.plot()
print(gdf)

B:\GitRepos\onhm-fetcher-parser\notebooks
..\Data
       POI_ID  hru_id_nat  hru_id_reg region  \
0     7733855           1           1     01   
1     7733919           2           2     01   
2     7732571           3           3     01   
3     7732387           4           4     01   
4     7733327           5           5     01   
...       ...         ...         ...    ...   
2457   719280        2458        2458     01   
2458   805345        2459        2459     01   
2459   806793        2460        2460     01   
2460   807073        2461        2461     01   
2461   725382        2462        2462     01   

                                               geometry  
0     POLYGON ((-73.37148 41.11233, -73.37151 41.112...  
1     POLYGON ((-73.38532 41.13467, -73.38566 41.134...  
2     POLYGON ((-73.41947 41.16068, -73.41955 41.160...  
3     POLYGON ((-73.41097 41.15833, -73.41105 41.158...  
4     MULTIPOLYGON (((-73.28636 41.12781, -73.28645 ...  
...                      

The code below (used when all hru shapefiles are read) is not optimum because it's base on the centroid of the hrus.  Therefore these boundary values are better and cover hrus outside the conus -126 54 -65 23

In [3]:
type(gdf['geometry'].bounds['minx'])
west = gdf['geometry'].bounds['minx'].min()
east = gdf['geometry'].bounds['maxx'].max()
south = gdf['geometry'].bounds['miny'].min()
north = gdf['geometry'].bounds['maxy'].max()
print(west, north, east, south)

-73.73815293899997 48.100042571000074 -66.94953853699997 40.98519585200006


#### Some example URLs for for daymet
 
https://thredds.daac.ornl.gov/thredds/ncss/grid/daymet-v3-agg/na.ncml/dataset.html

https://thredds.daac.ornl.gov/thredds/ncss/daymet-v3-agg/na.ncml?var=lat&var=lon&var=dayl&var=prcp&var=srad&var=swe&var=tmax&var=tmin&var=vp&disableLLSubset=on&disableProjSubset=on&horizStride=1&time_start=2019-09-22T12%3A00%3A00Z&time_end=2018-12-31T12%3A00%3A00Z&timeStride=1&accept=netcdf

In [4]:
# import requests
# from requests.exceptions import HTTPError
# from datetime import datetime, timedelta
# from urllib.parse import urlencode

# prcpurl = 'https://thredds.daac.ornl.gov/thredds/ncss/daymet-v3-agg/na.ncml'
# prcppayload = {
# #     'var': 'lat&var=lon&var=tmax',
#     'var': 'lat&var=lon&var=prcp&var=srad&var=swe&var=tmax&var=tmin&var=vp',
#     'north': '54',
#     'west': '-126',
#     'east': '-65',
#     'south': '20',
#     'disableProjSubset': 'on',
#     'horizStride': '1',
#     'time_start': '2018-12-31T12:00:00Z',
#     'time_end': '2018-12-31T12:00:00Z',
#     'timeStride': '1',
#     'accept': 'netcdf'}    
# try:
#     s = requests.Session()
#     #https://github.com/psf/requests/issues/1454
#     qry = urlencode(prcppayload).replace('%26','&')
#     qry = qry.replace('%3D', '=')
#     print(qry)
#     tmaxfile = requests.get(prcpurl, params=qry)
#     tmaxfile.raise_for_status()
# except HTTPError as http_err:
#     print(f'HTTP error occured: {http_err}')
# except Exception as err:
#     print(f'Other error occured: {err}')
# else:
#     print('Gridmet data retrieved!')

In [5]:
# with open('tmax_test2.nc', 'wb') as fh:
#     fh.write(tmaxfile.content)
# fh.close

In [6]:
#=========================================================
#            MACAV2METDATA FILE PARAMETERS
#=========================================================
# dirPath='https://thredds.daac.ornl.gov/thredds/ncss/ornldaac/1328/2018/daymet_v3_prcp_2018_na.nc4?var=lat&var=lon&var=prcp&north=52.880049298000074+&west=-124.72462483099997+&east=-66.94953853699997+&south=24.839424370000074&disableLLSubset=on&disableProjSubset=on&horizStride=1&time_start=2018-12-31T00:00:00Z&time_end=2018-12-31T00:00:00Z&timeStride=1&accept=netcdf'
# fileName='/thredds/dodsC/MET/tmmx/tmmx_2019.nc'
dirPath = 'tmax_test2.nc'
# dirPath = 'daymet_v3_tmax_2018_na.nc4.nc'
#--------------------------------------------------------
#   FORM FILENAME AND GET HANDLE TO FILE AND DATA
#--------------------------------------------------------
fullfilename= dirPath
print(fullfilename)

ds = xr.open_dataset(fullfilename)

print(ds)

# df = ds.to_dataframe()

print('\n The meta data is: \n', ds.attrs)
lathandle=ds['lat']
lonhandle=ds['lon']
timehandle=ds['time']
datahandle=ds['tmax']
dhlat = ds['lat']
dhlon = ds['lon']
crshandle=ds['lambert_conformal_conic']
print('\n The crs meta data is \n', crshandle.attrs)
print(datahandle)
# crstransform = crshandle.attrs['GeoTransform']
# print(crstransform)

#collect data to describe geotransform
lonmin = float(ds.attrs['geospatial_lon_min'])
latmax = float(ds.attrs['geospatial_lat_max'])
# lonres = float(ds.attrs['geospatial_lon_resolution'])
# latres = float(ds.attrs['geospatial_lon_resolution'])

#Print some information on the data

print('\n Data attributes, sizes, and coords \n') 
print('\n Data attributes are: \n',datahandle.attrs)
print('\n Data sizes are: \n', datahandle.sizes)
print('\n Data coords are: \n', datahandle.coords)
print('\n Lat coords are: \n', dhlat.attrs)

ts = datahandle.sizes
print(type(ts))
print(ts['time'])
dayshape = ts['time']
Lonshape = ts['x']
Latshape = ts['y']
#dayshape,lonshape,latshape = datahandle.values.shape
print(dayshape, Lonshape, Latshape)

# datahandle.values[dayshape-1,:,:].shape

# print(lathandle.values.shape)
# print(type(lathandle.values))
# print(datahandle.dtype)
# print(np.isfortran(datahandle.values))

tmax_test2.nc
<xarray.Dataset>
Dimensions:                  (time: 1, x: 6008, y: 3699)
Coordinates:
  * y                        (y) float32 1687.0 1686.0 ... -2010.0 -2011.0
  * x                        (x) float32 -2754.25 -2753.25 ... 3251.75 3252.75
  * time                     (time) datetime64[ns] 2018-12-31
Data variables:
    lat                      (y, x) float32 ...
    lambert_conformal_conic  int16 ...
    lon                      (y, x) float32 ...
    prcp                     (time, y, x) float32 ...
    srad                     (time, y, x) float32 ...
    swe                      (time, y, x) float32 ...
    tmax                     (time, y, x) float32 ...
    tmin                     (time, y, x) float32 ...
    vp                       (time, y, x) float32 ...
Attributes:
    _NCProperties:       version=1|netcdflibversion=4.4.1|hdf5libversion=1.8.17
    start_year:          1980
    source:              Daymet Software Version 3.0
    Version_software:    Daymet S

In [7]:
print(np.min(dhlat.values))

18.55824


In [ ]:
ds.variables.keys()

In [ ]:
tmax = ds.variables['tmax'][:]  # Reads the whole array
tmax

In [ ]:
ds.tmax.plot()

In [ ]:
# f1, ax1 = plt.subplots(1)
# ax1.set_aspect('equal')
# # ax1.set(xlim=(-130, -60), ylim=(20, 55))
# ptmax = ds.tmax
# ptmax_1 = ptmax.isel(time=0)
# print(type(ptmax_1))
# print(ptmax_1.shape)
# # ptmax_1.plot(ax=ax1, x='x', y='y', cmap='viridis')
# ds.tmax.plot(ax=ax1, x='x', y='y')
# f1.tight_layout()
# f1.savefig('daymet.png')

In [ ]:
for i in range(1,10): print(i)

In [8]:
lon = ds.lon.values
lat = ds.lat.values
print(np.shape(lon)[0], np.shape(lon)[1])

3699 6008


In [9]:
np.shape(lon)[0]-1

3698

In [10]:
tindex = []
tlat = []
tlon = []
count = 0
for i in range(1, np.shape(lon)[0]-1):
    if i%100 == 0: print(i)
    for j in range(1, np.shape(lon)[1]-1):
        tindex.append(count)
        tlat.append(lat[i,j])
        tlon.append(lon[i,j])
        count+=1
        
tdf = pd.DataFrame({'index':tindex,
                   'lat':tlat,
                   'lon':tlon})

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600


In [11]:
tdf.head()

,index,lat,lon
0,0,51.521744,-143.052292
1,1,51.526348,-143.039291
2,2,51.530949,-143.026276
3,3,51.535549,-143.013275
4,4,51.540150,-143.000259


In [12]:
from shapely.geometry import Polygon
def distance(p1x, p1y, p2x, p2y):
    return np.sqrt(np.power((p2x-p1x),2) + np.power((p2y-p1y),2))

#first create dataframe with temp
# df = pd.DataFrame({'temperature': ds.tmax.values.flatten()})
# res = 0.04166666/2.0
numcells2 = (np.shape(lat)[0]-2)*(np.shape(lat)[1]-2) # -2 to ignore boundaries, daymet domain should well overlap conus
poly2 = []
index2 = np.zeros(numcells2)
count = 0
# ncfcell = gpd.GeoDataFrame()
# ncfcell['geometry'] = None

for i in range(1, np.shape(lon)[0]-1):
    if i%100 == 0: print(i)
    for j in range(1, np.shape(lon)[1]-1):
        
        tpoly_1_lon = [lon[i,j], lon[i,j-1], lon[i+1,j-1], lon[i+1, j]]
        tpoly_1_lat = [lat[i,j], lat[i,j-1], lat[i+1,j-1], lat[i+1, j]]
        tpoly_1 = Polygon(zip(tpoly_1_lon, tpoly_1_lat))
        p1 = tpoly_1.centroid
        
        tpoly_2_lon = [lon[i,j], lon[i+1,j], lon[i+1,j+1], lon[i, j+1]]
        tpoly_2_lat = [lat[i,j], lat[i+1,j], lat[i+1,j+1], lat[i, j+1]]
        tpoly_2 = Polygon(zip(tpoly_2_lon, tpoly_2_lat))
        p2 = tpoly_2.centroid  
        
        tpoly_3_lon = [lon[i,j], lon[i,j+1], lon[i-1,j+1], lon[i-1, j]]
        tpoly_3_lat = [lat[i,j], lat[i,j+1], lat[i-1,j+1], lat[i-1, j]]
        tpoly_3 = Polygon(zip(tpoly_3_lon, tpoly_3_lat))
        p3 = tpoly_3.centroid  

        tpoly_4_lon = [lon[i,j], lon[i-1,j], lon[i-1,j-1], lon[i, j-1]]
        tpoly_4_lat = [lat[i,j], lat[i-1,j], lat[i-1,j-1], lat[i, j-1]]
        tpoly_4 = Polygon(zip(tpoly_4_lon, tpoly_4_lat))
        p4 = tpoly_4.centroid  
        
        lon_point_list = [p1.x, p2.x, p3.x, p4.x]
        lat_point_list = [p1.y, p2.y, p3.y, p4.y]
        
        poly2.append(Polygon(zip(lon_point_list, lat_point_list)))
        index2[count] = count
        count += 1
# ncfcells = gpd.GeoDataFrame(df, index=index, crs=ds['crs'], geometry=poly)  
ncfcells2 = gpd.GeoDataFrame(tdf, index=index2, geometry=poly2)  
ncfcells2.head()
print('Creating Spatial Index - This could take some time')
spatial_index2 = ncfcells2.sindex
print('Finished Spatial Index')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
Creating Spatial Index - This could take some time
Finished Spatial Index


In [ ]:
import csv
tcount = 0
vhruid = 1217
with open('tmp2_weights2t.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for index, row in gdf.iterrows():
#         print(index, row)
        if np.int(row['hru_id_nat']) == vhruid :
            count = 0
            if tcount == 0:
                writer.writerow(['grid_ids', 'hru_id_nat', 'w'])
            possible_matches_index = list(spatial_index2.intersection(row['geometry'].bounds))
            if not(len(possible_matches_index) == 0):
                possible_matches = ncfcells2.iloc[possible_matches_index]
                precise_matches = possible_matches[possible_matches.intersects(row['geometry'])]
                if not(len(precise_matches) == 0):
                    res_intersection = gpd.overlay(gdf.loc[[index]], precise_matches, how='intersection')
                    for nindex, row in res_intersection.iterrows():
                        ttt = gdf.loc[[index], 'geometry'].area
                        tmpfloat = np.float(res_intersection.area.iloc[nindex]/ttt[index])
                        writer.writerow([np.int(precise_matches.index[count]), np.int(row['hru_id_nat']), tmpfloat])
                        count += 1
                    tcount += 1
                    if tcount%100 == 0:
                        print(tcount, index)
        
            else:
                print('no intersection: ', index, np.int(row['nhm_id']))
            break

In [ ]:
import csv
tcount = 0
vhruid = 1217
with open('tmp2_weights2t.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for index, row in gdf.iterrows():
#         print(index, row)
        if np.int(row['hru_id_nat']) == vhruid :
            count = 0
            if tcount == 0:
                writer.writerow(['grid_ids', 'hru_id_nat', 'w'])
            possible_matches_index = list(spatial_index2.intersection(row['geometry'].bounds))
            if not(len(possible_matches_index) == 0):
                possible_matches = ncfcells2.iloc[possible_matches_index]
                precise_matches = possible_matches[possible_matches.intersects(row['geometry'])]
                if not(len(precise_matches) == 0):
                    res_intersection = gpd.overlay(gdf.loc[[index]], precise_matches, how='intersection')
                    for nindex, row in res_intersection.iterrows():
                        ttt = gdf.loc[[index], 'geometry'].area
                        tmpfloat = np.float(res_intersection.area.iloc[nindex]/ttt[index])
                        writer.writerow([np.int(precise_matches.index[count]), np.int(row['hru_id_nat']), tmpfloat])
                        count += 1
                    tcount += 1
                    if tcount%100 == 0:
                        print(tcount, index)
        
            else:
                print('no intersection: ', index, np.int(row['nhm_id']))
            break

In [ ]:
res_intersection


In [ ]:
ax = res_intersection.plot(cmap='tab10')
ax = gdf.loc[[index], 'geometry'].plot(ax=ax, facecolor='none', edgecolor='k')
precise_matches.plot(ax=ax, facecolor='none', edgecolor='k')
# index = 0
count = 0
cumwght = 0
for nindex, row in res_intersection.iterrows():
    ttt = gdf.loc[[index], 'geometry'].area
    tmpfloat = np.float(res_intersection.area.iloc[nindex]/ttt[index])
    # writer.writerow([np.int(precise_matches.index[count]), np.int(row['GFv11_id']), np.int(row['hru_id_nat']), tmpfloat])
    #writer.writerow([np.int(precise_matches.index[count]), np.int(row['GFv11_id']), tmpfloat])
    cumwght+=tmpfloat
    if nindex == 0:
        print('daymet_index', 'hru_index', 'intersection_area', 'hru_area', 'weight', 'cumulative_weight')
    print(np.int(precise_matches.index[count]), np.int(row['hru_id_nat']), res_intersection.area.iloc[nindex], ttt[index], tmpfloat, cumwght)
    count += 1
print(res_intersection.area, np.sum(res_intersection.area), gdf.loc[[index], 'geometry'].area)

In [ ]:
import csv
tcount = 0
with open('tmp2_weights2t.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for index, row in gdf.iterrows():
        count = 0
        if tcount == 0:
            writer.writerow(['grid_ids', 'hru_id_nat', 'w'])
        possible_matches_index = list(spatial_index2.intersection(row['geometry'].bounds))
        if not(len(possible_matches_index) == 0):
            possible_matches = ncfcells2.iloc[possible_matches_index]
            precise_matches = possible_matches[possible_matches.intersects(row['geometry'])]
            if not(len(precise_matches) == 0):
                res_intersection = gpd.overlay(gdf.loc[[index]], precise_matches, how='intersection')
                for nindex, row in res_intersection.iterrows():
                    ttt = gdf.loc[[index], 'geometry'].area
                    tmpfloat = np.float(res_intersection.area.iloc[nindex]/ttt[index])
                    writer.writerow([np.int(precise_matches.index[count]), np.int(row['hru_id_nat']), tmpfloat])
                    count += 1
                tcount += 1
                if tcount%100 == 0:
                    print(tcount, index)
        
            else:
                print('no intersection: ', index, np.int(row['nhm_id']))


In [ ]:
from numpy.ma import masked
# add tmax column to dataframe
gdf['tmax']=0.0
# gdf.tmax[noint] = 10.0
# print(gdf.tmax[noint])

wght_UofI = pd.read_csv('tmp2_weights2t.csv')
print(wght_UofI.head())

#iterate through hru's, grab all weights associated with hru_id, get total weighted value from netcdf file, assign to tmax
# ndata = datahandle.values[dayshape-1,:,:].flatten(order='C')
# ndata = ds.tmax.values[dayshape-1,:,:].flatten()
lon = ds.lon.values
lat = ds.lat.values
ndata = np.zeros(np.shape(lon)[1]*np.shape(lon)[0])
tlc = 0
for i in range(1, np.shape(lon)[0]-1):
    if i%10 == 0: print(i)
    for j in range(1, np.shape(lon)[1]-1):
        ndata[tlc] = datahandle.values[dayshape-1,i,j]
        tlc+=1

# print(ndata[1000:])
hruid = 'GFv11_id'
hruid = 'hru_id_nat'
unique_hru_ids = wght_UofI.groupby(hruid)
print(len(gdf), len(unique_hru_ids))

def np_get_wval2(grp, ndata):
    mdata = np.ma.masked_array(ndata[grp['grid_ids'].values.astype(int)], np.isnan(ndata[grp['grid_ids'].values.astype(int)]))
    return np.ma.average(mdata, weights=grp['w'])
def np_get_wval(ndata, wghts, hru_id):
    """
    Returns weighted average of ndata with weights = grp
    1) mdata = the subset of values associated with the gridmet id's that are mapped to hru_id.
    2) Some of these values may have nans if the gridmet id is outside of conus so only return values
    that are inside of conus
    3) this means that hru's that are entirely outside of conus will return nans which will ultimately,
    outside of this function get assigned zero's.
    4) the value is assigned the weighted average
    :param ndata: float array of data values
    :param wghts: float array of weights
    :param hru_id hru id number
    :return: numpy weighted averaged - masked to deal with nans associated with
            ndata that is outside of the conus.
    """
    mdata = np.ma.masked_array(ndata[wghts['grid_ids'].values.astype(int)],
                               np.isnan(ndata[wghts['grid_ids'].values.astype(int)]))

    # mdata = np.ma.masked_where(ndata[wghts['grid_ids'].values.astype(int)] <= 0.0,
    #                            (ndata[wghts['grid_ids'].values.astype(int)]))
    tmp = np.ma.average(mdata, weights=wghts['w'])
    if tmp is masked:
        # print('returning masked value', hru_id, mdata, wghts['w'])
        return np.nan

    else:
        return tmp
    
# unique_hru_ids.get_group(gdf['hru_id_nat']).agg({'tmax': np_get_wval(weight_id_rows, ndata)})    
td = np.zeros(len(gdf.index))
for index, row in gdf.iterrows():
#     if not(row['nhm_id'] in noint):
    try: 
        weight_id_rows = unique_hru_ids.get_group(row[hruid])
        td[index] = np.nan_to_num(np_get_wval(ndata, weight_id_rows, index+1))
    except KeyError:
        td[index] = 0.0
            
            
#         gdf['tmax'][index] = np_get_wval2(weight_id_rows, ndata)
#     else:
#         td[index] = 10.0

gdf['tmax'] = td.tolist()
# gdf['tmax'].fillna(0.0)

print('min/max', gdf['tmax'].min(), gdf['tmax'].max())

f, ax = plt.subplots(2, figsize=(12,12))
gdf.plot(ax=ax[0], column = 'tmax',linewidth=0., edgecolor='white')
ax[0].set_aspect('equal')
ptmax = ds.tmax
ptmax_1 = ptmax.isel(time=dayshape-1)
lvs = np.arange(gdf['tmax'].min(), gdf['tmax'].max(), 0.5)
ptmax_1.plot(ax=ax[1], levels=lvs, cmap='viridis')

In [ ]:
f, ax = plt.subplots(2, figsize=(12,12))
gdf.plot(ax=ax[0], column = 'tmax',linewidth=0., edgecolor='white')
ptmax = ds.tmax
ptmax_1 = ptmax.isel(time=dayshape-1)
lvs = np.arange(gdf['tmax'].min(), gdf['tmax'].max(), 0.5)
ptmax_1.plot(ax=ax[1], levels=lvs, cmap='viridis')
# print(temp)
# print(temp.lat)
hru_1 = ptmax_1.where((ds.lon>=-74) & (ds.lon<=-67) & (ds.lat>=40.5) & (ds.lat<=48.5), drop=True)
# print(delaware)
lvs = np.arange(gdf['tmax'].min(), gdf['tmax'].max(), 0.5)
p=hru_1.plot(ax=ax[1], levels=lvs, cmap='viridis')
ax[1].set_aspect('equal','box')